In [ ]:
##data_handler.py

class OddsDataHandler:
    def __init__(self):
        self.events = []  # A list to store odds data (each event is a dictionary)

    def load_data(self, odds_data):
        """
        Load odds data into the handler.
        :param odds_data: List of dictionaries, each representing an event
        """
        self.events = odds_data

    def get_events(self):
        """
        Retrieve all loaded events.
        :return: List of all events
        """
        return self.events

    def filter_events_by_sport(self, sport_key):
        """
        Filter events by sport (e.g., 'basketball_nba').
        :param sport_key: Sport key to filter by
        :return: Filtered list of events
        """
        return [event for event in self.events if event.get("sport_key") == sport_key]

        # Longer version:
        # filtered_events = []
        # for event in self.events:
        #     if event.get("sport_key") == sport_key:
        #         filtered_events.append(event)
        # return filtered_events

    def filter_events_by_team(self, team_name):
        """
        Filter events to find those involving a specific team.
        :param team_name: Name of the team to search for
        :return: Filtered list of events
        """
        return [event for event in self.events if team_name in event.get("teams", [])]

        # Longer version:
        # filtered_events = []
        # for event in self.events:
        #     teams = event.get("teams", [])
        #     if team_name in teams:
        #         filtered_events.append(event)
        # return filtered_events

    def filter_events_by_bookmaker(self, bookmaker):
        """
        Filter events to include only those with odds from a specific bookmaker.
        :param bookmaker: Name of the bookmaker (e.g., 'draftkings')
        :return: Filtered list of events
        """
        return [event for event in self.events if bookmaker in event.get("odds", {})]

        # Longer version:
        # filtered_events = []
        # for event in self.events:
        #     odds_dict = event.get("odds", {})
        #     if bookmaker in odds_dict:
        #         filtered_events.append(event)
        # return filtered_events

    def sort_events_by_odds(self, team_name, descending=True):
        """
        Sort events by the odds of a specific team.
        :param team_name: Name of the team
        :param descending: Sort in descending order (default: True)
        :return: Sorted list of events
        """
        return sorted(
            self.events,
            key=lambda event: event.get("odds", {}).get(team_name, float("-inf")),
            reverse=descending
        )

        # Longer version:
        # def get_team_odds(event):
        #     odds_dict = event.get("odds", {})
        #     return odds_dict.get(team_name, float("-inf"))
        #
        # return sorted(
        #     self.events,
        #     key=get_team_odds,
        #     reverse=descending
        # )

    def get_best_odds(self):
        """
        Get the best odds for each team across all bookmakers.
        :return: Dictionary of team -> best odds
        """
        best_odds = {}
        for event in self.events:
            for team, odds in event.get("odds", {}).items():
                if team not in best_odds or odds > best_odds[team]:
                    best_odds[team] = odds
        return best_odds

        # Longer version:
        # best_odds = {}
        # for event in self.events:
        #     odds_dict = event.get("odds", {})
        #     for team, odds in odds_dict.items():
        #         if team not in best_odds:
        #             best_odds[team] = odds
        #         elif odds > best_odds[team]:
        #             best_odds[team] = odds
        # return best_odds


# New Section

In [ ]:
class OddsCalculator:
    @staticmethod
    def american_to_decimal(american_odds):
        """
        Convert American odds to decimal odds
        :param american_odds: American odds (e.g., +150 or -150)
        :return: Decimal odds (e.g., 2.50 or 1.67)
        """
        try:
            if american_odds > 0:
                return (american_odds / 100) + 1
            else:
                return (100 / abs(american_odds)) + 1
        except (TypeError, ZeroDivisionError):
            raise ValueError("Invalid American odds value")

    @staticmethod
    def decimal_to_american(decimal_odds):
        """
        Convert decimal odds to American odds
        :param decimal_odds: Decimal odds (e.g., 2.50)
        :return: American odds (e.g., +150 or -150)
        """
        try:
            if decimal_odds <= 1:
                raise ValueError("Decimal odds must be greater than 1")

            if decimal_odds >= 2:
                return round((decimal_odds - 1) * 100)
            else:
                return round(-100 / (decimal_odds - 1))
        except (TypeError, ZeroDivisionError):
            raise ValueError("Invalid decimal odds value")

    @staticmethod
    def calculate_implied_probability(odds, odds_format="decimal"):
        """
        Convert odds to implied probability.
        :param odds: Odds value (decimal or American)
        :param odds_format: 'decimal' or 'american'
        :return: Implied probability (e.g., 0.555...)
        """
        try:
            if odds_format == "american":
                decimal_odds = OddsCalculator.american_to_decimal(odds)
            else:
                decimal_odds = float(odds)

            if decimal_odds <= 0:
                raise ValueError("Invalid odds value")

            return 1 / decimal_odds
        except (TypeError, ZeroDivisionError):
            raise ValueError("Invalid odds value")

    @staticmethod
    def calculate_no_vig_probabilities(implied_probs):
        """
        Remove the bookmaker's vig (margin) and calculate fair probabilities.
        :param implied_probs: Dictionary of team -> implied probability
        :return: No-vig probabilities
        """
        try:
            if not implied_probs:
                raise ValueError("Empty probabilities dictionary")

            total = sum(implied_probs.values())

            if total <= 0:
                raise ValueError("Total probabilities must be positive")

            return {team: prob / total for team, prob in implied_probs.items()}
        except (TypeError, ZeroDivisionError):
            raise ValueError("Invalid probability values")

    @staticmethod
    def calculate_vig(implied_probs):
        """
        Calculate the vig (juice) from a set of implied probabilities
        :param implied_probs: Dictionary of team -> implied probability
        :return: Vig as a percentage (e.g., 0.05 for 5% vig)
        """
        try:
            total_prob = sum(implied_probs.values())
            return total_prob - 1
        except (TypeError):
            raise ValueError("Invalid probability values")

    @staticmethod
    def convert_odds(odds, from_format="american", to_format="decimal"):
        """
        Convert odds between different formats
        :param odds: Odds value to convert
        :param from_format: Original format ('american' or 'decimal')
        :param to_format: Target format ('american' or 'decimal')
        :return: Converted odds value
        """
        if from_format == to_format:
            return odds

        if from_format == "american" and to_format == "decimal":
            return OddsCalculator.american_to_decimal(odds)
        elif from_format == "decimal" and to_format == "american":
            return OddsCalculator.decimal_to_american(odds)
        else:
            raise ValueError("Invalid odds format specified")


In [ ]:
##Tests

from data_handler import OddsDataHandler

# Sample mock data for testing
mock_odds_data = [
    {
        "event": "Warriors vs Nuggets",
        "sport_key": "basketball_nba",
        "teams": ["Warriors", "Nuggets"],
        "odds": {"draftkings": 1.80, "Nuggets": 2.10}
    },
    {
        "event": "Lakers vs Suns",
        "sport_key": "basketball_nba",
        "teams": ["Lakers", "Suns"],
        "odds": {"fanduel": 2.30, "Suns": 1.65}
    },
    {
        "event": "Chelsea vs Arsenal",
        "sport_key": "soccer_epl",
        "teams": ["Chelsea", "Arsenal"],
        "odds": {"betmgm": {"Chelsea": 2.50, "Arsenal": 2.75}}
    },
    {
        "event": "Real Madrid vs Barcelona",
        "sport_key": "soccer_laliga",
        "teams": ["Real Madrid", "Barcelona"],
        "odds": {"fanduel": {"Real Madrid": 1.85, "Barcelona": 2.05}}
    }
]

# Initialize the OddsDataHandler
data_handler = OddsDataHandler()

# Load the mock data
print("\nLoading mock data...")
data_handler.load_data(mock_odds_data)
print("Data loaded successfully!")

# Test: Get all events
print("\nAll Events:")
all_events = data_handler.get_events()
print(all_events)

# Test: Filter events by sport
print("\nFilter Events by Sport (basketball_nba):")
basketball_events = data_handler.filter_events_by_sport("basketball_nba")
print(basketball_events)

# Test: Filter events by team
print("\nFilter Events by Team (Warriors):")
warriors_events = data_handler.filter_events_by_team("Warriors")
print(warriors_events)

# Test: Filter events by bookmaker
print("\nFilter Events by Bookmaker (draftkings):")
draftkings_events = data_handler.filter_events_by_bookmaker("draftkings")
print(draftkings_events)

# Test: Sort events by odds
print("\nSort Events by Odds for Warriors (Descending):")
sorted_events = data_handler.sort_events_by_odds("Warriors", descending=True)
print(sorted_events)

# Test: Get best odds across all teams
print("\nBest Odds Across All Teams:")
best_odds = data_handler.get_best_odds()
print(best_odds)

